In [ ]:
!pip install ta

In [58]:
import pandas as pd
from ta import add_all_ta_features

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import yfinance as yf

In [59]:
start_date = "2017-01-01"
end_date = "2022-06-01"
symbol = "^VIX"
data = yf.download(symbol, start=start_date, end=end_date)
data.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,14.07,14.07,12.85,12.85,12.85,0
2017-01-04,12.78,12.80,11.63,11.85,11.85,0
2017-01-05,11.96,12.09,11.40,11.67,11.67,0
2017-01-06,11.70,11.74,10.98,11.32,11.32,0
2017-01-09,11.71,12.08,11.46,11.56,11.56,0


In [66]:
data = add_all_ta_features(data, open="Open", high="High", low="Low", close="Adj Close", volume="Volume", fillna=True)
data.shape
data.head()

,Open,High,Low,Close,Adj Close,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr,TARGET
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,14.07,14.07,12.85,12.85,12.85,0,-0.0,0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,12.850000,0.000000,0.000000,0.000000,-1
2017-01-04,12.78,12.80,11.63,11.85,11.85,0,-0.0,0,0.0,-0.0,...,-0.124879,-0.499515,0.0,0.0,0.0,12.098375,-7.782101,-8.101594,-7.782101,-1
2017-01-05,11.96,12.09,11.40,11.67,11.67,0,-0.0,0,0.0,-0.0,...,-0.345249,-0.881483,0.0,0.0,0.0,11.825780,-1.518990,-1.530645,-9.182881,-1
2017-01-06,11.70,11.74,10.98,11.32,11.32,0,-0.0,0,0.0,-0.0,...,-0.659566,-1.257265,0.0,0.0,0.0,11.503803,-2.999146,-3.045041,-11.906620,1
2017-01-09,11.71,12.08,11.46,11.56,11.56,0,-0.0,0,0.0,-0.0,...,-0.985604,-1.304152,0.0,0.0,0.0,11.534477,2.120148,2.097985,-10.038910,-1


In [65]:
data["TARGET"] = -1
data.loc[data["Adj Close"].shift(-1) > data["Adj Close"], "TARGET"] = 1
data.dropna(inplace=True)
data.head()

,Open,High,Low,Close,Adj Close,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr,TARGET
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,14.07,14.07,12.85,12.85,12.85,0,-0.0,0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,12.850000,0.000000,0.000000,0.000000,-1
2017-01-04,12.78,12.80,11.63,11.85,11.85,0,-0.0,0,0.0,-0.0,...,-0.124879,-0.499515,0.0,0.0,0.0,12.098375,-7.782101,-8.101594,-7.782101,-1
2017-01-05,11.96,12.09,11.40,11.67,11.67,0,-0.0,0,0.0,-0.0,...,-0.345249,-0.881483,0.0,0.0,0.0,11.825780,-1.518990,-1.530645,-9.182881,-1
2017-01-06,11.70,11.74,10.98,11.32,11.32,0,-0.0,0,0.0,-0.0,...,-0.659566,-1.257265,0.0,0.0,0.0,11.503803,-2.999146,-3.045041,-11.906620,1
2017-01-09,11.71,12.08,11.46,11.56,11.56,0,-0.0,0,0.0,-0.0,...,-0.985604,-1.304152,0.0,0.0,0.0,11.534477,2.120148,2.097985,-10.038910,-1


In [75]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [76]:
X_fs = StandardScaler().fit_transform(X)

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X_fs, y, test_size=0.7, random_state=42)

In [82]:
n_components = 5
pca = PCA(n_components=n_components)
pca_result = pca.fit(X_train)
X_train_pca = pca_result.transform(X_train)
X_test_pca = pca_result.transform(X_test)

In [83]:
print("Variance of each component: ", pca.explained_variance_ratio_)
print("\n Total Variance Explained: ", round(sum(list(pca.explained_variance_ratio_)) * 100, 2))

Variance of each component:  [0.35558763 0.27094564 0.10421284 0.04330093 0.0292655 ]

 Total Variance Explained:  80.33


In [84]:
pca_cols = []
for i in range(n_components):
    pca_cols.append(f"PC_{i}")

In [85]:
df_pca = pd.DataFrame(data=X_train_pca, columns=pca_cols)
df_pca.head()

,PC_0,PC_1,PC_2,PC_3,PC_4
0,-4.521072,2.350437,3.065682,-0.727829,-0.702189
1,3.625300,3.998660,2.736633,-0.744728,0.227143
2,-4.838797,-3.876852,-0.315338,2.138117,-0.205529
3,0.281133,5.380549,-2.451955,-1.934704,0.965231
4,-4.774590,-6.697827,-2.448823,2.030890,-1.487191
